In [ ]:
import random
import pickle

import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer
import tensorflow as tf


from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import LSTM,Dense ,Activation
from tensorflow.keras.optimizers import RMSprop


In [ ]:
url = 'https://raw.githubusercontent.com/lutzhamel/fake-news/master/data/fake_or_real_news.csv'
df = pd.read_csv(url)


In [ ]:
text = list(df.text.values)
joined_text = " ".join(text)

In [ ]:
partial_text = joined_text[:10000]

In [ ]:
partial_text=partial_text.lower()

In [ ]:
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(partial_text)

In [ ]:
unique_tokens = np.unique(tokens)

In [ ]:
unique_tokens_index = {token:idx for idx,token in enumerate(unique_tokens)}

In [ ]:
n_words = 10
input_words = []
next_words = []
for i in range(len(tokens)- n_words ):
  input_words.append(tokens[i:i + n_words])
  next_words.append(tokens[i + n_words])


In [ ]:
X = np.zeros((len(input_words),n_words,len(unique_tokens)), dtype=bool)
Y = np.zeros((len(next_words),len(unique_tokens)),dtype=bool)

In [ ]:
for i,words in enumerate(input_words):
  for j,word in enumerate(words):
    X[i,j,unique_tokens_index[word]] =1
  Y[i,unique_tokens_index[next_words[i]]] =1

In [ ]:
model = Sequential()
model.add(LSTM(128,input_shape=(n_words,len(unique_tokens)),return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer=RMSprop(learning_rate=0.01),metrics=['accuracy'])
model.fit(X,Y,batch_size = 128,epochs=30,shuffle=True)

Epoch 1/30
14/14 [==============================] - 3s 9ms/step - loss: 5.1747 - accuracy: 0.0795
Epoch 2/30
14/14 [==============================] - 0s 8ms/step - loss: 4.8307 - accuracy: 0.1007
Epoch 3/30
14/14 [==============================] - 0s 7ms/step - loss: 4.5867 - accuracy: 0.1035
Epoch 4/30
14/14 [==============================] - 0s 6ms/step - loss: 4.3531 - accuracy: 0.1230
Epoch 5/30
14/14 [==============================] - 0s 6ms/step - loss: 4.0302 - accuracy: 0.1642
Epoch 6/30
14/14 [==============================] - 0s 6ms/step - loss: 3.7192 - accuracy: 0.1762
Epoch 7/30
14/14 [==============================] - 0s 6ms/step - loss: 3.3622 - accuracy: 0.2203
Epoch 8/30
14/14 [==============================] - 0s 6ms/step - loss: 3.0260 - accuracy: 0.2712
Epoch 9/30
14/14 [==============================] - 0s 6ms/step - loss: 2.6437 - accuracy: 0.3576
Epoch 10/30
14/14 [==============================] - 0s 6ms/step - loss: 2.2948 - accuracy: 0.4462
Epoch 11/30
14/14 [

In [ ]:
def pre_next_word(input_text,n_best):
  input_text = input_text.lower()
  X = np.zeros((1,n_words,len(unique_tokens)))
  for i,word in enumerate(input_text.split()):
    X[0,i,unique_tokens_index[word]]=1
  predictions = model.predict(X)[0]
  return np.argpartition(predictions,-n_best)[-n_best:]

In [ ]:
possible = pre_next_word("He will have to look into this thing and he",5)

1/1 [==============================] - 1s 813ms/step


In [ ]:
print([unique_tokens[idx] for idx in possible])

['wouldn', 'it', 'out', 'not', 'be']
